# Tutorial on using multiple models for evaluation
- This tutorial is an example of how to test multiple models on the openml data to see which one performs the best.
- The evaluation is still a bit basic, but it is a good starting point for future research.

In [1]:
from __future__ import annotations
from langchain_community.cache import SQLiteCache
import os
import sys
import chromadb
from pathlib import Path
from tqdm import tqdm

import pandas as pd
# change the path to the backend directory
sys.path.append(os.path.join(os.path.dirname("."), '../../backend/'))

In [2]:
from modules.utils import load_config_and_device
from modules.llm import *
from modules.results_gen import aggregate_multiple_queries_and_count

/Users/smukherjee/.pyenv/versions/3.10.14/envs/openml/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Setting the config

In [3]:
new_path = Path("../../backend/")

config = load_config_and_device(str(new_path / "config.json"), training = True)

config["type_of_data"] = "dataset"
config["training"] = True

[INFO] Finding device.
[INFO] Device found: mps


## Defining the models used
- Embedding models are any from Huggingface hub
- LLM models are any from Ollama library

In [4]:
# list_of_embedding_models = ["BAAI/bge-small-en-v1.5", "Alibaba-NLP/gte-Qwen2-1.5B-instruct"]
list_of_embedding_models = ["BAAI/bge-large-en-v1.5"]
list_of_llm_models = ["qwen2:1.5b", "phi3", "llama3"]

In [5]:
def process_embedding_model_name_hf(name : str) -> str:
    """
    Description: This function processes the name of the embedding model from Hugging Face to use as experiment name.
    
    Input: name (str) - name of the embedding model from Hugging Face.
    
    Returns: name (str) - processed name of the embedding model.
    """
    return name.replace("/", "_")

def process_llm_model_name_ollama(name : str) -> str:
    """
    Description: This function processes the name of the llm model from Ollama to use as experiment name.
    
    Input: name (str) - name of the llm model from Ollama.

    Returns: name (str) - processed name of the llm model.
    """
    return name.replace(":", "_")

## Defining the evaluation queries
- replace this with a proper dataframe for a more comprehensive evaluation

In [6]:
# queries = ["Find datasets related to COVID-19", "Find datasets related to COVID-19 and India", "COVID-19 dataset", "COVID-19 dataset India", "Mexico historical covid"]

## Downloading the models
- PLEASE MAKE SURE YOU HAVE DOWNLOADED OLLAMA (```curl -fsSL https://ollama.com/install.sh | sh```)

In [7]:
# download the ollama llm models

# os.system("curl -fsSL https://ollama.com/install.sh | sh")
os.system("ollama serve&")
print("Waiting for Ollama server to be active...")  
while os.system("ollama list | grep 'NAME'") == "":
    pass

for llm_model in list_of_llm_models:
    os.system(f"ollama pull {llm_model}")

Error: listen tcp 127.0.0.1:11434: bind: address already in use


Waiting for Ollama server to be active...
NAME         	ID          	SIZE  	MODIFIED      


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling 405b56374e02... 100% ▕████████████████▏ 934 MB                         
pulling 62fbfd9ed093... 100% ▕████████████████▏  182 B                         
pulling c156170b718e... 100% ▕████████████████▏  11 KB                         
pulling f02dd72bb242... 100% ▕████████████████▏   59 B    

## Running the steps
- Create an experiment directory
- Save a config file with the models and the queries in the experiment directory
- Download openml data for each dataset and format into a string
- Create vectorb and embed the data
- Get the predictions for each model for a list of queries and evaluate the performance
- (note) At the moment, this runs for a very small subset of the entire data. To disable this behavior and run on the entire data, set ```config["test_subset_2000"] = False```

### Override setup_vector_db_and_qa to use a list of IDs instead of all of them

In [8]:
def setup_vector_db_and_qa(
    config: dict, data_type: str, client: ClientAPI, subset_ids: list = None
):
    """
    Description: Create the vector database using Chroma db with each type of data in its own collection. Doing so allows us to have a single database with multiple collections, reducing the number of databases we need to manage.
    This also downloads the embedding model if it does not exist. The QA chain is then initialized with the vector store and the configuration.

    Input: config (dict), data_type (str), client (chromadb.PersistentClient)

    Returns: qa (langchain.chains.retrieval_qa.base.RetrievalQA)
    """

    config["type_of_data"] = data_type

    # Download the data if it does not exist
    openml_data_object, data_id, all_metadata, handler = get_all_metadata_from_openml(
        config=config
    )
    # Create the combined metadata dataframe
    metadata_df, all_metadata = create_metadata_dataframe(
        handler, openml_data_object, data_id, all_metadata, config=config
    )

    # subset the metadata if subset_ids is not None
    if subset_ids is not None:
        metadata_df = metadata_df[metadata_df["did"].isin(subset_ids)]

    # Create the vector store
    vectordb = load_document_and_create_vector_store(
        metadata_df, config=config, chroma_client=client
    )
    # Initialize the LLM chain and setup Retrieval QA
    qa = initialize_llm_chain(vectordb=vectordb, config=config)
    return qa, all_metadata

In [9]:
# use a tiny subset of the data for testing (no need for the edited subset)

In [10]:
use_custom_data = False


if use_custom_data:
    config["test_subset"] = False
else:
    config["test_subset"] = True

for embedding_model in tqdm(list_of_embedding_models, desc="Embedding Models", total=len(list_of_embedding_models)):
    for llm_model in tqdm(list_of_llm_models, desc="LLM Models", total=len(list_of_llm_models)):
        # update the config with the new embedding and llm models
        config["embedding_model"] = embedding_model
        config["llm_model"] = llm_model

        # create a new experiment directory using a combination of the embedding model and llm model names
        experiment_name = f"{process_embedding_model_name_hf(embedding_model)}_{process_llm_model_name_ollama(llm_model)}"
        experiment_path = new_path/Path(f"../data/experiments/{experiment_name}")

        # create the experiment directory if it does not exist
        os.makedirs(experiment_path, exist_ok=True)
       
        # update the config with the new experiment directories
        config["data_dir"] = str(experiment_path)
        config["persist_dir"] = str(experiment_path / "chroma_db")

        # save training details and config in a dataframe
        config_df = pd.DataFrame.from_dict(config, orient='index').reset_index()
        config_df.columns = ['Hyperparameter', 'Value']
        config_df.to_csv(experiment_path / "config.csv", index=False)

        # load the persistent database using ChromaDB
        client = chromadb.PersistentClient(path=config["persist_dir"])

        # Run "training"
        qa_dataset = setup_vector_db_and_qa(
            config=config, data_type=config["type_of_data"], client=client
        )
        
        # # Run an evaluation by aggregating multiple queries and counting the results
        # # TODO : Replace this evaluation with a more meaningful one
        combined_df = aggregate_multiple_queries_and_count(queries,qa_dataset=qa_dataset, config=config, group_cols = ["id", "name"], sort_by="query", count = False)

        # # TODO : ADD LLM evaluation here when the function is ready

        combined_df.to_csv(experiment_path / "results.csv")

Embedding Models:   0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Training is set to True.
[INFO] Subsetting the data.
[INFO] Initializing cache.
[INFO] Getting dataset metadata from OpenML.



QUEUEING TASKS | : 100%|██████████| 500/500 [00:00<00:00, 61440.60it/s]




PROCESSING TASKS | : 100%|██████████| 500/500 [00:00<00:00, 582.60it/s]

COLLECTING RESULTS | : 100%|██████████| 500/500 [00:00<00:00, 410803.53it/s]


[INFO] Saving metadata to file.
[INFO] Loading model...
[INFO] Model loaded.
[INFO] Generating unique documents. Total documents: 11676
Number of unique documents: 9684 vs Total documents: 11676





































Batches: 100%|██████████| 16/16 [01:03<00:00,  3.94s/it]



































Batches: 100%|██████████| 16/16 [00:57<00:00,  3.57s/it]



































Batches: 100%|██████████| 16/16 [01:06<00:00,  4.14s/it]



































Batches: 100%|██████████| 16/16 [01:10<00:00,  4.39s/it]



































Batches: 100%|██████████| 16/16 [01:05<00:00,  4.10s/it]













